# PRD: Inventory Analyzer v1.0

## Objective  
Enable warehouse managers and supply chain planners at MiMcoffee to monitor, analyze, and optimize inventory levels in real time, reduce stockouts and overstock, improve procurement decisions, and ensure traceability—using data from the existing mimcoffee_warehouse.db SQLite database.

## Scope  

- Read-only analytics from the existing SQLite database (mimcoffee_warehouse.db)
- Product-level stock tracking (including Citrus Blend Coffee)


In [10]:
from course_code.common.utils import stdio_mcp
from agentscope.mcp import StdIOStatefulClient

import os.path
sqlite_db_name = "mimcoffee_warehouse.sqlite"
sqlite_db_path=os.path.join(os.getcwd(),"..","..","src","mimcoffee","databases",sqlite_db_name)
sqlite_db_directory = os.path.dirname(sqlite_db_path)

stdio_json={
  "sqlite": {
    "command": "uvx",
    "args": [
      "mcp-server-sqlite",
      "--db-path",
      sqlite_db_path
    ]
  }
}
mcp_client = StdIOStatefulClient(
    **stdio_mcp(stdio_json)
)
await mcp_client.connect()

2025-11-04 15:22:47,477 | INFO    | _stateful_client_base:connect:66 - MCP client connected.


In [11]:
# construct the prompt according to the PRD.
system_prompt="""You are tasked with building a desktop-based Inventory Analyzer application for MiMcoffee, a specialty coffee company. The tool must read from an existing SQLite database (mimcoffee_warehouse.db) and provide actionable insights into inventory levels, trends, and alerts—without modifying the database.

Database Schema:
The database includes the following tables:

products (with fields: product_id, product_code, name, category, unit)
inventory (current stock per product)
purchases (inflow records in 2025)
sales_out (outflow records in 2025)
inventory_log (full audit trail of stock changes)
"""
prompt ="""
What is the current inventory of Citrus Blend Coffee?
"""

In [15]:
from course_code.common.utils import load_env
from agentscope.memory import InMemoryMemory
from agentscope.agent import ReActAgent
from agentscope.tool import Toolkit
from agentscope.formatter import OpenAIChatFormatter
from agentscope.message import Msg
from agentscope.model import OpenAIChatModel
from course_code.common.constants import MODELSCOPE_ENDPOINT,QWEN_MODEL_NAME

load_env()
# construct the toolkit
toolkit = Toolkit()
# register the search mcp client in toolkit
await toolkit.register_mcp_client(
    mcp_client
)

# build the  agent in the ReAct 
agent = ReActAgent(
    # The name of the Agent
    name="MiMCoffeeInventoryAnalyzer",
    # The system prompt we prepared before.
    sys_prompt=system_prompt,
    # An OpenAI-compatible model
    model=OpenAIChatModel(
        # The API key of the ModelScope
        api_key=os.environ.get("MODELSCOPE_API_KEY"),
        # The model name
        model_name=QWEN_MODEL_NAME,
        # The args to declare the Endpoint
        client_args={
            "base_url":MODELSCOPE_ENDPOINT
        },
        
        stream=False,
    ),
    formatter=OpenAIChatFormatter(),
    toolkit=toolkit,
    memory=InMemoryMemory(),
)

# run the agent
reply = await agent( Msg("User",prompt,role="user"))

2025-11-04 15:23:21,161 | INFO    | _toolkit:register_mcp_client:716 - Registered 6 tool functions from MCP: read_query, write_query, create_table, list_tables, describe_table, append_insight.
2025-11-04 15:23:23,789 | WARNING | _openai_formatter:_format:230 - Unsupported block type thinking in the message, skipped.


MiMCoffeeInventoryAnalyzer(thinking): 
{
    "type": "tool_use",
    "id": "call_623f80facf964dc28049e6",
    "name": "read_query",
    "input": {
        "query": "SELECT i.quantity FROM inventory i JOIN products p ON i.product_id = p.product_id WHERE p.name = 'Citrus Blend Coffee';"
    }
}

system: {
    "type": "tool_result",
    "id": "call_623f80facf964dc28049e6",
    "name": "read_query",
    "output": [
        {
            "type": "text",
            "text": "[{'quantity': 55.0}]"
        }
    ]
}
MiMCoffeeInventoryAnalyzer(thinking): 
MiMCoffeeInventoryAnalyzer: The current inventory of Citrus Blend Coffee is 55.0 units.
